In [ ]:
## 
import numpy as np
import pandas as pd
from scipy.spatial import Voronoi

import tyssue

from tyssue.core.objects import Epithelium
from tyssue.geometry.bulk_geometry import BulkGeometry as bgeom
from tyssue.core.generation import hexa_grid3d, from_3d_voronoi
from tyssue.draw.vispy_draw import vp_view


In [ ]:
grid = hexa_grid3d(6, 4, 3)

datasets = from_3d_voronoi(Voronoi(grid))
eptm = Epithelium('start3D', datasets)

gspecs = eptm.set_geom('sheet')
bgeom.update_all(eptm)
eptm.cell_df.head()

In [ ]:
datasets['face'].shape

In [ ]:
from tyssue.core import generation

In [ ]:
from tyssue.core.generation import hexa_grid2d, from_2d_voronoi
grid = hexa_grid2d(6, 4, 1, 1)
datasets = from_2d_voronoi(Voronoi(grid))


for key, df in datasets.items():
    print(key, df.shape)

In [ ]:
vp_view(eptm)

In [ ]:
bounds = [[-0.1, 2], [0, 2], [0, 10]]

edgeout = eptm.cut_out(bounds)

eptm.remove(edgeout)

In [ ]:
eptm.vert_df['z'] = eptm.vert_df['z'] * np.cos(eptm.vert_df['y'])
bgeom.update_all(eptm)

In [ ]:
import vispy.io as io

In [ ]:
bgeom.update_all(eptm)

In [ ]:
eptm.edge_df.head()

In [ ]:
io.write_mesh('test.obj', vertices=vertices, faces=faces,
              normals=eptm.edgedf[['nx', 'ny', 'nz']].values,
              texcoords=None, overwrite=True)